In [4]:
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# You will need the following three imported functions in this homework
from cryptography_functions import is_prime, modular_inverse, convert_to_text

# The following two lines of code give you an overview of the content of the 
# module cryptography_functions as a printout 
import cryptography_functions 
help(cryptography_functions)

# You can also simply inspect the file cryptography_functions.py in jupyter

ModuleNotFoundError: No module named 'cryptography_functions'

In [6]:
def eves_decompose(N):
# first set variables as required in part 1
    current_N = N
    current_decomp = {}
# iterate through p = 2,3,4,5.. etc
    for p in range(2,N):
# at stage p, if it is not the case that current_N > 1, return current_decomp
        if current_N <= 1:
            return current_decomp
# Otherwise at stage p, if p is prime, and current_N is divisible by p then, 
# during stage p - supposing that e is the greatest integer such that current_N 
# is divisible by p**e - the value of current_N is divided by p**e and the key,
# value pair p: e is added to current_decomp. The algorithm then proceeds to stage p + 1.
        if is_prime(p, num_wit=50) == True:
            if pow(current_N,1,p) == 0:
                power = False
                e = 2
                while not power:
                    if pow(current_N,1,pow(p,e)) == 0:
                        e += 1   
                    else:  
                        e -= 1
                        power = True
                current_N = current_N // p**e
                current_decomp[p] = e  
    return current_decomp

In [7]:
# This cell is for your  own tests and rough work
# Here's an easy example. 
K = 3**4 * 17**2 * 23*5 * 37*1
eves_decompose(K)

NameError: name 'is_prime' is not defined

In [8]:
# Testing area, using a 1030 bit number N as input  
N = int('''
83706564861599899653596475840424730458044600907771647007334050372336
09892280960094075716733849713226033323457029628024552980324599333370
00350684058261595028326433198358461905686764348015800175804034215489
64422648032179381441885225061637776302818897970437444034711022068655
35792022518811024903672288999022155493'''.replace('\n',''))
N_dict = {8273: 6, 9173: 4, 10847: 4, 11311: 4, 11621: 4, 11953: 1, 
          12479: 2, 12923: 3, 13331: 4, 17209: 6, 17827: 1, 22643: 3, 
          25693: 2, 28279: 5, 30059: 3, 40763: 1, 41149: 4, 42589: 6, 
          55529: 1, 56299: 3, 56659: 5}
# Your function should pass this test (and a hidden test)
assert eves_decompose(N) == N_dict
# There is one hidden test using a similarly large number as input

NameError: name 'is_prime' is not defined

In [9]:
# Function from Lecture 3 of Week 10 for your information. 
def rsa_decrypt(c,p,q,N,e): 
    '''
    Given input (c,p,q,N,e) returns the RSA decryption of ciphertext
    c using private key (p,q) and public key (N,e). (We input N as a 
    parameter to avoid having to recompute N = p*q.)
    '''
    totient = N - (p + q) + 1       # This is (p-1)*(q-1)
    f = modular_inverse(e,totient)  # Note: f * e = 1 (mod totient)
    return pow(c,f,N)               # This is m = c**f (mod N) where c = m**e

In [10]:
def eves_decrypt(c,N_decomp,N,e):
    t = 1
# now calculate totient
    for i in N_decomp.keys():
        t = t * i**(N_decomp[i]-1) * (i-1)
# now calculate modular inverse
    f = modular_inverse(e,t)
    return pow(c,f,N)

In [11]:
# This cell is for your  own tests and rough work
# An easy example its easy to come up with others... 
N = 3**4 * 13**6 * 19**2 * 53**8 
N_decomp = eves_decompose(N)
e = 65537 
m = 53458936789543654   
c = pow(m,e,N)
m_new = eves_decrypt(c,N_decomp,N,e)
m_new # This should be the same as m (test this using an assert statement)

NameError: name 'is_prime' is not defined

In [12]:
# Testing area using large integers. 
# The ciphertext transmitted by Bob is c = m**e (mod N1) for some m. 
c = int('''
44040993505419961079485909296703212068236418038487210630216568513457921
38336402714823913994482821220533646724844746539049498179119183080513149
31408838044424151820320215981324447571417572264574845615546729136893812
49131244618204208416492118915226940262437591117873293250397449419056765
862894569167485210222637280212955530851785230'''.replace('\n',''))
# Alice's public key is (N1,e). 
e = 65537
N1 = int('''
10230611115167627701967833246600529852209634787210723189242636200865741
18444371403714267935391853062750090843150978074427150943772945195279171
00280972327473328434991888940897882646502852180501590304145861856226481
21545694830420218027450318764061548271599311224935464351312228384098752
0276059163143000753961230876956414639120841839'''.replace('\n',''))
# The prime decomposition of N1 is N1_decomp. 
N1_decomp = {8237: 1, 8377: 6, 10343: 1, 12941: 4, 14563: 3, 14939: 1, 
            15991: 7, 16691: 5, 20011: 1, 20627: 1, 22039: 5, 22111: 2,
            25633: 7, 26189: 5, 32603: 6, 50159: 7, 52259: 3, 54721: 3, 
            62383: 7}
m_decrypt = 30786253411008024821942804353038798977485124256255665198
# This is true - i.e. c = m_decrypt**e (mod N1) - since m_decrypt == m
assert c == pow(m_decrypt,e,N1)
# This is the test of your function 
assert eves_decrypt(c,N1_decomp,N1,e) == m_decrypt

NameError: name 'modular_inverse' is not defined

In [13]:
# calculate prime decomposition
def eves_hack(c,N,e,verbose=False):
    N_decomp = eves_decompose(N)
# find m
    m = eves_decrypt(c,N_decomp,N,e)
# find message
    message = convert_to_text(m)
    if verbose:
        print('Firstly:\nc = {},\nN = {},\ne = {}'.format(c,N,e))
        print('\nWe then decompsed N to a dictionary of its primes, {}'.format(N_decomp))
        print('\nNext we decrypted c by calculating its totient, thus finding m={}'.format(m))
        print('\nUsing the convert_to_text algorithm, we transfered m into string, giving the message:')
    return message

In [14]:
# The output of this cell - i.e. the printout generated by your function 
# (since verbose=True below) will be hand graded.

# The ciphertext transmitted by Bob is c2. 
c2 = int('''
13765857727387973232479930349817413548909598342616602193701631284325656267
80701134620002785255145626349800021032002509266271558856266934655194975687
59307106770371716547021307786879941777280361623176540814670229006368058329
84911920670968761587938063429816954180041385102041952823773836621293529317
2747559925006181429813854'''.replace('\n',''))

# Alice's public key is (N2,e). 
e = 65537
N2 = int('''
45826872776221747553965304038569114157321898032045936109324798759092548505
31613377090447584756132759059985898756678241036420212547809537827143541861
23768838634031792685862961283953201133968017357971449925795092740936013316
34869263468578803287892075018530064166921208113922227263664667767248577655
3724508783300940735066371
'''.replace('\n',''))

# Eve now hacks ciphertext c2
eves_hack(c2,N2,e,True)

NameError: name 'is_prime' is not defined